In [1]:
true_json = """{
    [
    {
        "shape": "cone",
        "size": "small",
        "material": "rubber",
        "color": "cyan",
    },
    {
        "shape": "cube",
        "size": "small",
        "material": "metallic",
        "color": "cyan",
       
    },
        {
        "shape": "cylinder",
        "size": "large",
        "material": "metallic",
        "color": "red",
       
    }
]}"""

prediction_json = """{
    [
    {
        "shape": "cube",
        "size": "small",
        "material": "metallic",
        "color": "cyan",
       
    },
       {
        "shape": "cone",
        "size": "small",
        "material": "rubber",
        "color": "cyan",
    },

        {
        "shape": "cylinder",
        "size": "large",
        "material": "metallic",
        "color": "red",
       
    }
]}"""

prediction_json = """

"""



example_output= """
I'd be happy to help you score the predictions.

To match the objects from the Ground Truth (GT) to the Prediction json, I will use a simple approach: since the lists are unordered, I will try to find the best match for each object in the GT by comparing it with each object in the Prediction json. I will use a greedy approach, matching the first object in the GT with the first object in the Prediction json that has the most matching attributes, and so on.

Here's the matching process:

1. GT object 1: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Matched with: Prediction object 1: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Score: 4 (all attributes match)

2. GT object 2: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Matched with: Prediction object 2: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Score: 4 (all attributes match)

3. GT object 3: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Matched with: Prediction object 3: {"shape": "cone", "size": "large", "material": "metallic", "color": "cyan"}
   Score: 4 (all attributes match)

The total score is: 4 + 4 + 4 = 12

So, the final score is: {"score": 12}
"""

In [2]:
from utils.load_env_vars import load_env
load_env()

-----------------------------
Environment variables loaded:
HF_HOME /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
HUGGINGFACE_HUB_CACHE /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
WANDB_PROJECT llava-changes
-----------------------------


In [3]:
pred = """
The video features a minimalist scene with a gray surface and a simple background. It begins with a small purple cube positioned in the center, flanked by a small green cone on the left and a small blue cylinder on the right. The green cone moves closer to the purple cube, eventually passing by it and moving out of the frame to the left. The blue cylinder then moves closer to the purple cube, passing by it and moving out of the frame to the right. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame, moving closer to the purple cube before moving out of the frame again. The blue cylinder reappears on the right side of the frame, moving closer to the purple cube before moving out of the frame again. The green cone reappears on the left side of the frame,
"""

gt = """
[
    {
        "shape": "cube",
        "size": "small",
        "material": "metal",
        "color": "purple",
        "transformation": {
            "enlarges": "False",
            "rotates": "False",
            "orbits": "False",
            "change_colors": "False"
        }
    },
    {
        "shape": "cone",
        "size": "small",
        "material": "metal",
        "color": "cyan",
        "transformation": {
            "enlarges": "True",
            "rotates": "False",
            "orbits": 0,
            "change_colors": "blue"
        }
    },
    {
        "shape": "cylinder",
        "size": "small",
        "material": "metal",
        "color": "blue",
        "transformation": {
            "enlarges": "False",
            "rotates": "True",
            "orbits": "False",
            "change_colors": "False"
        }
    }
]
"""

# prompt

In [13]:
judge_prompt= """
Hey, can you help me score the following description? Given is the ground truth json and a scene description in natural language. Go through the description and try to match the objects from the GT to the prediction json and write down how you would match this. 
Then in a next step go through the ground truth json and annotate it true if the property is correct for the matched object and false if it is not.

For example you have the following scene description:
There is a small cyan metallic cone and asmall cyan metallic cube. The cone is orbiting around the cube.

And the following GT:
{{
    "shape": "cube",
    "size": "small",
    "material": "metallic",
    "color": "cyan",
    "transformation": {{
        "enlarges": "False",
        "rotates": "False",
        "orbits": "False",
        "change_colors": "False"
    }}
       
}},
{{
    "shape": "cone",
    "size": "small",
    "material": "rubber",
    "color": "cyan",
    "transformation": {{
        "enlarges": "False",
        "rotates": "False",
        "orbits": "True",
        "change_colors": "False"
        }}
}},

The matching would be:
GT object 1: {{"shape": "cube", "size": "small", "material": "metallic", "color": "cyan"}}
Matched with: small cyan metallic cube
GT object 2: {{"shape": "cone", "size": "small", "material": "rubber", "color": "cyan"}}
Matched with: small cyan metallic cone

Then in step 2 you would go through the GT and annotate the transformation properties as follows:
{{
    "shape": ["cube","correct"]
    "size": " ["small","correct"]
    "material": ["metallic","correct"]
    "color": ["cyan","correct"]
    "transformation": {{
        "enlarges": ["False","correct"]
        "rotates": ["False","correct"]
        "orbits": ["False","correct"]
        "change_colors": "["False""correct"]
        }} 
}},
{{
        "shape": ["cone","correct"]
        "size": ["small","correct"]
        "material": [rubber","false"]
        "color": ["cyan","correct"]
        "transformation": {{
            "enlarges": ["False","correct"]
            "rotates": ["False","correct"]
            "orbits": ["True","correct"]
            "change_colors": ["False", "correct"]
            }}
}},

I.e. put correct if the property is correct and false if it is not like in the rubbber material for the cone.

Here are the predictions and the GT you should work with:
Prediction:
{}

Ground Truth:
{}

Then finally you can write down the score for the description. For each property that is correct you get 1 point. So in the example above the score would be 7.
{{
    score: {{
        achieved: 7,
        total: 8}}
}}
"""

judge_prompt.format(pred, gt)

'\nHey, can you help me score the following description? Given is the ground truth json and a scene description in natural language. Go through the description and try to match the objects from the GT to the prediction json and write down how you would match this. \nThen in a next step go through the ground truth json and annotate it true if the property is correct for the matched object and false if it is not.\n\nFor example you have the following scene description:\nThere is a small cyan metallic cone and asmall cyan metallic cube. The cone is orbiting around the cube.\n\nAnd the following GT:\n{\n    "shape": "cube",\n    "size": "small",\n    "material": "metallic",\n    "color": "cyan",\n    "transformation": {\n        "enlarges": "False",\n        "rotates": "False",\n        "orbits": "False",\n        "change_colors": "False"\n    }\n       \n},\n{\n    "shape": "cone",\n    "size": "small",\n    "material": "rubber",\n    "color": "cyan",\n    "transformation": {\n        "en

In [14]:
import openai
client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="EMPTY")

def query_sglang_llama3(pred, gt):
    response = client.chat.completions.create(
        model="default",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant helping me score the predictions of a model."},
            {"role": "user",
            "content": judge_prompt.format(pred, gt)},
        ],
        temperature=0,
        max_tokens=700,
    )
    return response.choices[0].message.content

In [15]:
response = query_sglang_llama3(pred, gt)

print(response)


I'd be happy to help you score the prediction.

**Step 1: Matching objects from GT to prediction**

Based on the scene description, I match the objects from the Ground Truth (GT) to the prediction as follows:

GT object 1: {"shape": "cube", "size": "small", "material": "metal", "color": "purple"}
Matched with: small purple cube

GT object 2: {"shape": "cone", "size": "small", "material": "metal", "color": "cyan"}
Matched with: small green cone (note: color mismatch, expected cyan but got green)

GT object 3: {"shape": "cylinder", "size": "small", "material": "metal", "color": "blue"}
Matched with: small blue cylinder

**Step 2: Annotating transformation properties**

I annotate the transformation properties as follows:

{
    "shape": ["cube","correct"]
    "size": ["small","correct"]
    "material": ["metal","correct"]
    "color": ["purple","correct"]
    "transformation": {
        "enlarges": ["False","correct"]
        "rotates": ["False","correct"]
        "orbits": ["False","cor